In [ ]:
# IMPORT THE REQUIRED LIBRARIES

from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Output,
                        Model,
                        Metrics,
                        Markdown,
                        HTML,
                        component, 
                        OutputPath, 
                        InputPath)

from kfp import compiler
from google.cloud import aiplatform as vertex_
from google.cloud.aiplatform import pipeline_jobs

from datetime import datetime
import pandas as pd

In [ ]:
PROJECT_ID = "id"
REGION = 'us-central1'

BUCKET_NAME = PROJECT_ID+"-rb"
source_data_blob = "data/data.csv"
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipeline_root_rb/"


In [ ]:
print(PIPELINE_ROOT)

In [ ]:
# Custom base image created using docker

IMAGE_NAME = "training"
BASE_IMAGE = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/rb/{IMAGE_NAME}:latest"
print(BASE_IMAGE)

In [ ]:
import os
if "GOOGLE_APPLICATION_CREDENTIALS" not in os.environ:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = input("Enter path to Google Cloud service account key JSON: ")


In [ ]:
from google.cloud import storage


def create_bucket_class_location(bucket_name):
    """
    Create a new bucket in the US region with the coldline storage
    class
    """
    # bucket_name = "your-new-bucket-name"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    bucket.storage_class = "COLDLINE"
    new_bucket = storage_client.create_bucket(bucket, location="us-central1")

    print(
        "Created bucket {} in {} with storage class {}".format(
            new_bucket.name, new_bucket.location, new_bucket.storage_class
        )
    )
    return new_bucket


In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [ ]:
import io

def write_read(bucket_name, blob_name):
    """Write and read a blob from GCS using file-like IO"""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your new GCS object
    # blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    
    content = blob.download_as_bytes()

    df = pd.read_csv(io.BytesIO(content))
    return df

    

In [ ]:
df = write_read(BUCKET_NAME, "data/data.csv")

In [ ]:
df.head()

In [ ]:
#!pip install fsspec
#!pip install gcsfs

In [ ]:
df_train = pd.read_csv(f"gs://{BUCKET_NAME}/{source_data_blob}")

## Read the Dataset

In [ ]:
@component(
    base_image=BASE_IMAGE,
)

def get_std_data(
    gcs_bucket: str,
    gcs_path: str,
    dataset_train: Output[Dataset],
):
    # Set the service account key file for authentication
    
    import pandas as pd
    from google.cloud import storage
    # print(f"gs://{gcs_bucket}/{gcs_path}")
    df = pd.read_csv(f"gs://{gcs_bucket}/{gcs_path}")

    df.to_csv(dataset_train.path, index=False)

## Data Preprocessing

In [ ]:
@component(
    base_image=BASE_IMAGE,
)

def preprocess_std_data(
    train_df: Input[Dataset],
    dataset_train_preprocessed: Output[Dataset],
):
    
    import pandas as pd
    from src.prepare_data.preprocessing import data_preprocessing_pipeline
   
    train_df = pd.read_csv(train_df.path)
    
    train_df_preprocessed = data_preprocessing_pipeline(train_df)
    
    train_df_preprocessed.to_csv(dataset_train_preprocessed.path, index=False)

## Train Test Split

In [ ]:
@component(
    base_image=BASE_IMAGE,
)
def train_test_split(dataset_in: Input[Dataset],
                     dataset_train: Output[Dataset],
                     dataset_test: Output[Dataset],
                     test_size: float = 0.33):

    import pandas as pd
    from sklearn.model_selection import train_test_split

    df = pd.read_csv(dataset_in.path)
    df_train, df_test = train_test_split(df, test_size=test_size, random_state=42)

    df_train.to_csv(dataset_train.path, index=False)
    df_test.to_csv(dataset_test.path, index=False)

## Train the Model

In [ ]:
@component(
    base_image=BASE_IMAGE,
)
def train_busyness(
    dataset_train: Input[Dataset],
    dataset_test: Input[Dataset],
    model: Output[Model]
):
    
    import pandas as pd
    import pickle
    from src.train.model import BusynessEstimation
    from src.train.config import Data
    from src.utils.utils import get_image_data
    
    
    # Read train and test data
    train_data = pd.read_csv(dataset_train.path)
    test_data = pd.read_csv(dataset_test.path)
    
    # Instantiate the model class
    busyness_model = BusynessEstimation(
                                        test_data.copy()
                                        )
                                        
    # Create X_train and y_train
    X_train = train_data.drop(Data.target, axis=1)
    y_train = train_data[Data.target]

    # Fit the model (training pipeline consists of feature engineering, feature selection and training an xgboost model)
    busyness_model.fit(X_train, y_train)
    
    # Save the best hyperparameters as an artifact
    with open(best_params.path, "w") as f:
        f.write(str(busyness_model.best_params))
    """    
    shap.summary_plot(busyness_model.shap_values, busyness_model.X_test_transformed, max_display=20) # plot shap summary plot
    shap_plot_dataurl = get_image_data() # get image data to render the image in the html file
    html_content = f'<html><head></head><body><h1>Shap Summary Plot</h1>\n<img src={shap_plot_dataurl} width="97%"></body></html>' 
    # Save shap summary plot as an html artifact
    with open(shap_summary_plot.path, "w") as f: 
        f.write(html_content)
    """
    model.metadata["framework"] = "scikit-learn" 
    # Save the model as an artifact
    with open(model.path, 'wb') as f: 
        pickle.dump({
            "pipeline": busyness_model.model_pipeline,
            "target": busyness_model.target,
            "scores_dict": busyness_model.scores}, f)

## Evaluate the Model

In [ ]:
@component(
    base_image=BASE_IMAGE,
)
def evaluate_busyness(
    busyness_model: Input[Model],
    metrics_baseline: Output[Metrics],
    metrics_train: Output[Metrics],
    metrics_test: Output[Metrics]):
    
    import pickle
    
    file_name = busyness_model.path
    with open(file_name, 'rb') as file:  
        model_data = pickle.load(file)
    
    scores = model_data["scores_dict"] 

    def log_metrics(scores, metric):
        for metric_name, val in scores.items():
            metric.log_metric(metric_name, float(val))
            
    log_metrics(scores["baseline_scores"], metrics_baseline)        
    log_metrics(scores["train_scores"], metrics_train)
    log_metrics(scores["test_scores"], metrics_test)

## Deploy the Model

In [ ]:
@component(
    base_image=BASE_IMAGE,
    install_kfp_package=False,
)
def deploy_busyness(
        serving_container_image_uri: str,
        display_name: str,
        model_endpoint: str,
        gcp_project: str,
        gcp_region: str,
        model: Input[Model],
        vertex_model: Output[Model],
        vertex_endpoint: Output[Model]
):
    from google.cloud import aiplatform as vertex_ai
    from pathlib import Path
    
    # Checks existing Vertex AI Enpoint or creates Endpoint if it is not exist.
    def create_endpoint ():
        endpoints = vertex_ai.Endpoint.list(
        filter='display_name="{}"'.format(model_endpoint),
        order_by='create_time desc',
        project=gcp_project,
        location=gcp_region,
        )
        if len(endpoints) > 0:
            endpoint = endpoints[0] # most recently created
        else:
            endpoint = vertex_ai.Endpoint.create(
                display_name=model_endpoint,
                project=gcp_project,
                location=gcp_region
        )
        return endpoint

    endpoint = create_endpoint()
    
    # Uploads trained model to Vertex AI Model Registry or creates new model version into existing uploaded one.
    def upload_model ():
        listed_model = vertex_ai.Model.list(
        filter='display_name="{}"'.format(display_name),
        project=gcp_project,
        location=gcp_region,
        )
        if len(listed_model) > 0:
            model_version = listed_model[0] # most recently created
            model_upload = vertex_ai.Model.upload(
                    display_name=display_name,
                    parent_model=model_version.resource_name,
                    artifact_uri=str(Path(model.path).parent),
                    serving_container_image_uri=serving_container_image_uri,
                    location=gcp_region,
                    serving_container_predict_route="/predict",
                    serving_container_health_route="/health"
            )
        else:
            model_upload = vertex_ai.Model.upload(
                    display_name=display_name,
                    artifact_uri=str(Path(model.path).parent),
                    serving_container_image_uri=serving_container_image_uri,
                    location=gcp_region,
                    serving_container_predict_route="/predict",
                    serving_container_health_route="/health"
            )
        return model_upload
    
    uploaded_model = upload_model()
    
    # Save data to the output params
    vertex_model.uri = uploaded_model.resource_name

    # Deploys trained model to Vertex AI Endpoint
    model_deploy = uploaded_model.deploy(
        machine_type='n1-standard-1',
        endpoint=endpoint,
        traffic_split={"0": 100},
        deployed_model_display_name=display_name,
    )

    # Save data to the output params
    vertex_endpoint.uri = model_deploy.resource_name

## Create Pipeline

In [ ]:
# USE TIMESTAMP TO DEFINE UNIQUE PIPELINE NAMES
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DISPLAY_NAME = 'pipelinebusyness-job{}'.format(TIMESTAMP)

In [ ]:


@dsl.pipeline(
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="pipeline-regionsbusyness"   
)

def pipeline(
    data_filepath: str = "data/data.csv",
    data_bucket : str = BUCKET_NAME,
    project: str = PROJECT_ID,
    region: str = REGION    
):

    data_op = get_std_data(gcs_bucket = data_bucket, gcs_path = data_filepath )
    data_preprocess_op = preprocess_std_data(train_df = data_op.outputs["dataset_train"])
    train_test_split_op = train_test_split(dataset_in = data_preprocess_op.outputs["dataset_train_preprocessed"])
    train_model_op = train_busyness(dataset_train = train_test_split_op.outputs["dataset_train"], dataset_test = train_test_split_op.outputs["dataset_test"])
    model_evaluation_op = evaluate_busyness(busyness_model = train_model_op.outputs["model"])
    
    deploy_model_op = deploy_busyness(
        model = train_model_op.outputs['model'],
        gcp_project = project,
        gcp_region = region, 
        serving_container_image_uri = serving_container_image_uri,
        display_name = "rb",
        model_endpoint = "rb_endpoint"
    )
    


## Compile and Run the pipelie

In [ ]:
# COMPILE THE PIPELINE (to create the job spec file)

compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='ml_rb.json')

In [ ]:
# CREATE A RUN USING THE JOB SPEC FILE GENERATED 

start_pipeline = pipeline_jobs.PipelineJob(
    display_name="rb-pipeline",
    template_path="ml_rb.json",
    enable_caching=False,
    location=REGION,
)

In [ ]:
# RUN THE PIPELINE

start_pipeline.run()

## Make Predictions Using Vertex AI Endpoint

In [ ]:
from google.cloud import aiplatform as vertex_ai

endpoint_name = <ENDPOINT_URI>
endpoint = vertex_ai.Endpoint(endpoint_name)

In [ ]:
test_df = pd.read_csv('./data/test.csv')

In [ ]:
request = test_df.to_json(orient='records', lines=True)

In [ ]:
predictions = endpoint.predict(instances=request.splitlines())

In [ ]:
print(predictions)

In [ ]:
from kfp import compiler

compiler.Compiler().compile(get_std_data, package_path='get_data.yaml')
compiler.Compiler().compile(preprocess_std_data, package_path='preprocessing.yaml')
compiler.Compiler().compile(train_test_split, package_path='train_test_split.yaml')
compiler.Compiler().compile(train_busyness, package_path='model_training.yaml')
compiler.Compiler().compile(evaluate_busyness, package_path='model_evaluation.yaml')
compiler.Compiler().compile(deploy_busyness, package_path='model_deployment.yaml')
